# Project Design Writeup



### Project Problem and Hypothesis

* The aim of this project is to develop a time series regression analysis to predict the number of international tourists arriving in New York in a given month.
* The model will be used to predict a discrete number using supervised learning with the predictor variables being time (month and year) and exchange rates for the most relevant currencies (at least the Euro, and additonal ones TBD).
* Finding a successful model could help private and public organizations to plan for increases or decreases in the number of tourists visiting the city from one month to the next. For example, a tour company could have a better idea of how many guides they need on staff based on the expected number of international tourists. 
* I think the seasonal factor of the time of year(Summer vs Fall) will have the biggest impact and on a lesser level the measure of the exchange rate in the previous month for relevant currencies will be the best predictors for the number of international visitors.


### Datasets
* I have access to monthly international visitors (not including from Canada and Mexico) arriving in New York City airports (New York and Newark combined) going back to 2007. This is in the form of various Excel spreadsheets, so I need to combine the data together into one data frame. The numbers are cumulative from month to month, so I will need to take the differences using ".diff" function between different months once I have the data frame. Additionally I will need to append additional columns that contain the exchange rates for the USD compared to relevant currencies.
* The code below shows my attempt to merge the different existing Excel spreadsheets into a data frame.I ran into an issue becaus the different spreadsheets contain multiple tabs. I found out that I could convert each tab into a separate csv file. However, I could not figure out how to do this other than manually. Since there are 7 spreadhsheets with 12 tabs each, this would be a pretty tedious process. I need to determine if there is a more efficient way of doing this.


In [2]:
import numpy as np
import pandas as pd

In [3]:
cd ../../../Desktop/Data Science Final Project/Monthly arrivals

/Users/mikaelroussel/Desktop/Data Science Final Project/Monthly arrivals


In [4]:
ls

top_ports-2007.xls  top_ports-2009.xls  top_ports-2011.xls  top_ports-2013.xls
top_ports-2008.xls  top_ports-2010.xls  top_ports-2012.xls


In [5]:
pwd

u'/Users/mikaelroussel/Desktop/Data Science Final Project/Monthly arrivals'

In [6]:
import glob
glob.glob("*.xls")



['top_ports-2007.xls',
 'top_ports-2008.xls',
 'top_ports-2009.xls',
 'top_ports-2010.xls',
 'top_ports-2011.xls',
 'top_ports-2012.xls',
 'top_ports-2013.xls']

In [41]:
monthly_arrivals_1 = pd.DataFrame()
for f in glob.glob("*.xls"):
    monthly_arrivals_2 = pd.read_excel(f)
    monthly_arrivals = monthly_arrivals_1.append(monthly_arrivals_2,ignore_index=True)


In [42]:
monthly_arrivals.describe()

,U.S. PORT OF ENTRY (ALL MODES) OF NON-U.S. RESIDENT ARRIVALS,Unnamed: 1,Unnamed: 2,Unnamed: 3
count,24,19,18.000000,16
unique,24,19,18.000000,16
top,January YTD 2013,2013,0.001877,14
freq,1,1,1.000000,1


In [43]:
monthly_arrivals.head()

,U.S. PORT OF ENTRY (ALL MODES) OF NON-U.S. RESIDENT ARRIVALS,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,NaN,NaN,NaN,NaN
1,January YTD 2013,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,2013,NaN,NaN
4,NaN,TOTAL,% change,NaN


In [18]:
monthly_arrivals[:]

,U.S. PORT OF ENTRY (ALL MODES) OF NON-U.S. RESIDENT ARRIVALS,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,NaN,NaN,NaN,NaN
1,January YTD 2013,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,2013,NaN,NaN
4,NaN,TOTAL,% change,NaN
5,PORTS,OVERSEAS*,2013/2012,Rank
6,GRAND TOTAL,2073242,0.0654669,NaN
7,"MIAMI, FL",350304,0.134507,1
8,"NEW YORK, NY",291296,0.0895354,2
9,"LOS ANGELES, CA",210824,0.00187712,3


### Domain knowledge

* I have studied International Economics. This has some relevance given that I want to include exchange rates a predictor variable, although this is a pretty simple analysis, so I don't think a deep understanding of currency markets is needed. 
* While doing an internet search it seems like the main organization trying to predict visitor numbers to New York City is NYC & Company, the official tourism marketing organization for the city. I found a predicted number for total visitors in 2016. This is different from the model I'm working on because I will focus on international tourists on a monthly basis.

### Project Concerns

* At this point, I need to combine all the different spreadsheets into a usable data frame. I would like to figure out an efficient way to read the different tabs within the spreadsheets.
* A major assumption I am making is that international visitors arriving in New York City airports are actually spending some time in the city and not just flying through. I am also assuming that the same airports have been used to count the number of visitors arriving (i.e JFK and Newark during the whole timeframe I am looking at).
 * I wish I could predict the total number of visitors (both foreign and domestic) but I do not have monthly information for the number of domestic tourists to New York City.
* The risk of the model being wrong is not very high. At worst, organizations will not plan resources well to deal with the number of international tourists.
* There is a chance of especially the later monthly visitor numbers being incorrect and later being revised. There is a warning about this on the source website.

### Outcomes

* I expect the output to be a predicted number for the last month available for international tourists arriving to New York City. I will include all previous months probably going back to 2007 to train the model. I think I will also include a comparison of a prediction that only took into account the seasonal variable in the visitors number to see if including the exchange rates in the previous month created a more accurare prediction. 
 * There will also be a summary table evaluating the model like R squared, the coefficient values for the exchange rates, correlation and the p-value.
 * A graph showing the historic trend in the visitors numbers will be part of the output as well.
* I don't think my target audience would have any expectation other than a basic description of the variables used (historic numbers and exchange rates) as well as how successul the prediction was.
* I would expect a 1 unit change in exchange rate (gain of foreign currency(ies) vs. USD)would result in a 20,000 increase in the number of arrivals.
* The model developed could be considered successful if both the time variable and at least one exchange rate have coefficients that are statistically significant at the 95% confidence level.
* If the project is not successful in terms of developing a strong predictive model, then I will see if removing certain predictive variables helps. If this does not work, I will conclude that I used the wrong variables and hypothesize about what variables would be better to use.